# The Modified Benedict-Webb-Rubin equation of state 
Johnson, Zllweg, Gubbins, 
"The Lennard-Jones equation of state revisited",
Molecular Physics, 1993

implementing the parameters from Table 10

In [1]:
import numpy as np

In [2]:
par = np.loadtxt('mbwr_eos.dat')

$$ P = \sum_{n=1}^9 a_n \rho^n + \exp\left(\frac{-\rho^2}{\rho_c^2}\right)\sum_{n=10}^{15}\rho^{2n-17} $$

32 parameters:
  - 15: $a_1,\dots,a_{15}$
  - : $\rho_c$
  
**Reduced Units:**
  - residual Helmholtz free energy: $A^*_r = A_r/N\epsilon$
  - pressure: $P^* = P\sigma^3/\epsilon$